# Example Scraping and Vectorization of Wikipedia Article

In [50]:
import requests

category = 'Rare_diseases'
r = requests.get('https://en.wikipedia.org/w/api.php?action=query&format=json&list=categorymembers&cmtitle=Category:%s&cmcontinue=' % category)
content = r.json()
content

{u'batchcomplete': u'',
 u'continue': {u'cmcontinue': u'page|4142444f4d494e414c204550494c45505359|6935618',
  u'continue': u'-||'},
 u'query': {u'categorymembers': [{u'ns': 0,
    u'pageid': 189092,
    u'title': u'Rare disease'},
   {u'ns': 0, u'pageid': 47313691, u'title': u'13q deletion syndrome'},
   {u'ns': 0, u'pageid': 3821104, u'title': u'2-Hydroxyglutaric aciduria'},
   {u'ns': 0, u'pageid': 10947132, u'title': u'3C syndrome'},
   {u'ns': 0, u'pageid': 28203137, u'title': u'3q29 microdeletion syndrome'},
   {u'ns': 0, u'pageid': 3823832, u'title': u'48, XXXX'},
   {u'ns': 0, u'pageid': 3824105, u'title': u'49, XXXXX'},
   {u'ns': 0, u'pageid': 28202023, u'title': u'8p23.1 duplication syndrome'},
   {u'ns': 0, u'pageid': 7966521, u'title': u'Aarskog\u2013Scott syndrome'},
   {u'ns': 0, u'pageid': 3062597, u'title': u'ABCD syndrome'}]}}

In [51]:
page_ids = [item['pageid'] for item in content['query']['categorymembers']]
page_ids

[189092,
 47313691,
 3821104,
 10947132,
 28203137,
 3823832,
 3824105,
 28202023,
 7966521,
 3062597]

In [52]:
r2 = requests.get('https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&rvprop=content&pageids=%s' % "|".join(map(str, page_ids)))
content = r2.json()
content

{u'batchcomplete': u'',
 u'query': {u'pages': {u'10947132': {u'ns': 0,
    u'pageid': 10947132,
    u'revisions': [{u'*': u'{{Infobox disease\n | Name           = 3C syndrome\n | Image          = \n | Caption        = \n | DiseasesDB     = \n | ICD10          = {{ICD10|Q|87|0|q|80}}\n | ICD9           = <!--{{ICD9|xxx}}-->\n | ICDO           = \n | OMIM           = 220210\n | MedlinePlus    = \n | eMedicineSubj  = \n | eMedicineTopic = \n | MeshID         = \n}}\n\n\'\'\'3C syndrome\'\'\', also known as \'\'\'CCC dysplasia\'\'\', \'\'\'Craniocerebellocardiac dysplasia\'\'\'<ref>{{RareDiseases|5666}}</ref> or \'\'\'Ritscher\u2013Schinzel syndrome\'\'\',<ref name="Orpha">{{cite web |url = http://www.orpha.net/consor/cgi-bin/OC_Exp.php?lng=EN&Expert=7 |title = 3C syndrome |work = Orphanet |accessdate = 11 April 2014}}</ref> is a rare condition, whose symptoms include [[heart defects]], [[cerebellar hypoplasia]], and [[cranial dysmorphism]]. It was first described in the medical literature

In [53]:
page_content = [page['revisions'][0]['*'] for _, page in content['query']['pages'].iteritems()]
page_content

[u'\'\'\'13q deletion syndrome\'\'\' is a rare genetic disease caused by the deletion of some or all of the large arm of [[human chromosome 13]]. It causes intellectual disability and congenital malformations that affect a variety of organ systems.\n\n== Signs and symptoms ==\nDifferent areas of deletion are associated with different symptoms. Deletions from the centromere to 13q32 or any deletions including the 13q32 band are associated with slow growth, intellectual disability, and congenital malformations. Deletions from 13q33 to the end of the chromosome are associated with intellectual disability. Intellectual disabilities range from very mild to very severe, and can co-occur with [[behavioral disorder|behavioral disorders]] and/or [[autism spectrum]] disorders.<ref name=":0">{{Cite web|title = 13q deletion syndrome|url = http://www.socialstyrelsen.se/rarediseases/13qdeletionsyndrome|website = www.socialstyrelsen.se|accessdate = 2015-07-22}}</ref>\n\nAt birth, the main symptoms in

In [56]:
from gensim.corpora import wikicorpus

wiki_text = page_content
cleaned_text = [wikicorpus.filter_wiki(text).replace("== References ==", " ").replace("==", " ").replace("\n", " ") for text in wiki_text]
cleaned_text

[u"'''13q deletion syndrome''' is a rare genetic disease caused by the deletion of some or all of the large arm of human chromosome 13. It causes intellectual disability and congenital malformations that affect a variety of organ systems.    Signs and symptoms   Different areas of deletion are associated with different symptoms. Deletions from the centromere to 13q32 or any deletions including the 13q32 band are associated with slow growth, intellectual disability, and congenital malformations. Deletions from 13q33 to the end of the chromosome are associated with intellectual disability. Intellectual disabilities range from very mild to very severe, and can co-occur with behavioral disorders and/or autism spectrum disorders.  At birth, the main symptoms include low weight (due to intrauterine growth restriction), hypotonia, and feeding difficulties. Infants may also have cleft palate.  13q deletion syndrome gives a characteristic appearance to affected individuals, potentially includin

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english', binary=True)
X_train_counts = count_vect.fit_transform(cleaned_text)
X_train_counts.toarray()

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])